# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121977e+02     1.400676e+00
 * time: 0.6149749755859375
     1     1.070991e+01     7.692837e-01
 * time: 1.6808021068572998
     2    -1.148385e+01     9.195028e-01
 * time: 2.1826331615448
     3    -3.394439e+01     7.455609e-01
 * time: 2.9229609966278076
     4    -4.735230e+01     5.763491e-01
 * time: 3.6634740829467773
     5    -5.679472e+01     1.993491e-01
 * time: 4.412461996078491
     6    -5.969035e+01     1.571699e-01
 * time: 4.907142162322998
     7    -6.087557e+01     6.807928e-02
 * time: 5.402333974838257
     8    -6.138390e+01     3.828060e-02
 * time: 5.897899150848389
     9    -6.166870e+01     3.060372e-02
 * time: 6.394008159637451
    10    -6.183261e+01     2.759102e-02
 * time: 6.975162982940674
    11    -6.198671e+01     2.359237e-02
 * time: 7.471026182174683
    12    -6.207699e+01     1.346409e-02
 * time: 7.9664061069488525
    13    -6.211782e+01     1.378365e-02
 * time: 8.461764097213745
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671078
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522633
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336824

    total               -62.261666459579